<a href="https://colab.research.google.com/github/SuzaKrish/Computer-Vision/blob/master/DataAugmentation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Data_Aug'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/Data_Aug'
/content


In [ ]:
import numpy as np
import scipy
import csv
import cv2
import glob
from os import path
import random
import shutil
from PIL import Image, ImageOps


In [ ]:
def separate(): #
    #Extract the images for different classes and store in individual files 
    annotations = r'TrainAnnotation.csv' # Annotations pertaining to the training data
    count_cat = np.zeros([1, 5])
    source = r"TrainData-C2-20200408T163601Z-001/TrainData-C2//" #actual Training data

    with open(annotations, newline = '') as csvfile:
        csvread = csv.reader(csvfile)
        next(csvread)
        for row in csvread:
            if(row[1] == '0'):
                count_cat[0, 0] += 1
                if(path.isfile(path.join(source, row[0]))):
                    shutil.copy(path.join(source, row[0]), r'Cat0')
        
            elif(row[1] == '1'):
                count_cat[0, 1] += 1
                if(path.isfile(path.join(source, row[0]))):
                    shutil.copy(path.join(source, row[0]), r'Cat1')

            elif(row[1] == '2'):
                count_cat[0, 2] += 1
                if(path.isfile(path.join(source, row[0]))):
                    shutil.copy(path.join(source, row[0]), r'Cat2')

            elif(row[1] == '3'):
                count_cat[0, 3] += 1
                if(path.isfile(path.join(source, row[0]))):
                    shutil.copy(path.join(source, row[0]), r'Cat3')

            elif(row[1] == '4'):
                count_cat[0, 4] += 1
                if(path.isfile(path.join(source, row[0]))):
                    shutil.copy(path.join(source, row[0]), r'Cat4')


    print('The number of images in each category are: {}, {}, {}, {}, {}'.format(count_cat[0, 0], count_cat[0, 1], count_cat[0, 2], count_cat[0, 3], count_cat[0, 4]))
    return count_cat


In [ ]:
def flipy(img):
    return cv2.flip(img, 1)

In [ ]:
def blur(img):
    return cv2.GaussianBlur(img, (3,3), 2)

In [ ]:
def cropped(img):
    x = random.randint(0, img.shape[0]-10)
    y = random.randint(0, img.shape[1]-10)
    print(img.shape)
    window = 200
    right = y + window
    lower = x + window
    img1 = np.array(img[x : lower, y : right])
    img1 = cv2.resize(img1, (img.shape[1], img.shape[0]))
    print(img1.shape)
   
    return img1

In [ ]:
def enhance(img):
    x = img.shape[0]
    y = img.shape[1]
    ch = img.shape[2]
    b = 40
    new_img = np.zeros([img.shape[0], img.shape[1], img.shape[2]])
    for k in range(ch):
        for i in range(x):
            for j in range(y):
                new_img[i,j,k] = np.uint8(b * np.log(1 + img[i, j, k]))
    
    return new_img

#Data Augmentation :
###Bringing all categories to 500 images.Performs Flipping about y axis, Scaling, Blurring, Contrast Stretching, Rotation
###*c = Keeps track of new images created per Category
###*addlimit = Images to be added
###*When c == addlimit, stop augmenting
###*setinput = Keeps track of no.of old dataset images


In [ ]:
def aug(path, newpath, nc, cat):
  annotations = r'TrainAnnotation.csv'
  c = 0  # c = Keeps track of new images created per Category
  # When c == addlimit, stop augmenting 
    
  setinput = 0 # setinput = Keeps track of no.of old dataset images 

  for img in glob.glob(path):
    I = cv2.imread(img)
    setinput += 1
    if(setinput < 20 or (setinput >= 100 and setinput < 200)): # Works on original image
      setno = 0
      while(setno < 4):
        if setno == 0:
          newimg = cropped(I)
        elif setno == 1:
          newimg = flipy(I)
        elif setno == 2:
          newimg = blur(I)
        elif setno == 3:
          newimg = enhance(I)
                
        setno += 1
        c += 1
        nc += 1
        name = newpath + str(nc) + '.jpg'
        cv2.imwrite(name, newimg) 
        new_a = [name, cat]
        with open(annotations, 'a') as csvfile:
          csvapp = csv.writer(csvfile)
          csvapp.writerow(new_a)
        
    elif((setinput >= 20 and setinput < 40) or (setinput >= 200 and setinput < 300)): #Works on flipped image
      setno = 0
      while(setno < 4):
        if setno == 0:
          newimg = flipy(I)
          dup = newimg
        elif setno == 1:
          newimg = cropped(dup)
        elif setno == 2:
          newimg = blur(dup)
        elif setno == 3:
          newimg = enhance(dup)
                
        setno += 1
        c += 1
        nc += 1
        name = newpath + str(nc) + '.jpg'
        cv2.imwrite(name, newimg) 
        new_a = [name, cat]
        with open(annotations, 'a') as csvfile:
          csvapp = csv.writer(csvfile)
          csvapp.writerow(new_a)
            
    elif((setinput >= 40 and setinput < 80) or (setinput >= 300 and setinput < 400)): # Works on blurred image
      setno = 0
      while(setno < 4):
        if setno == 0: 
          newimg = blur(I)
          dup = newimg
        elif setno == 1:
         newimg = cropped(dup)
        elif setno == 2:
          newimg = flipy(dup)
        elif setno == 3:
          newimg = enhance(dup)
      setno += 1
      c += 1
      nc += 1
      name = newpath + str(nc) + '.jpg'
      cv2.imwrite(name, newimg) 
      new_a = [name, cat]
      with open(annotations, 'a') as csvfile:
        csvapp = csv.writer(csvfile)
        csvapp.writerow(new_a)

    elif((setinput >= 80 and setinput < 100) or (setinput >= 400 and setinput < 500)): # Works on enhanced image
      setno = 0 
      while(setno < 4):
        if setno == 0: 
          newimg = enhance(I)
          dup = newimg
        elif setno == 1:
          newimg = cropped(dup)
        elif setno == 2:
          newimg = flipy(dup)
        elif setno == 3:
          newimg = blur(dup)
                
      setno += 1
      c += 1
      nc += 1
      name = newpath + str(nc) + '.jpg'
      cv2.imwrite(name, newimg) 
      new_a = [name, cat]
      with open(annotations, 'a') as csvfile:
        csvapp = csv.writer(csvfile)
        csvapp.writerow(new_a)
            
    c5 = cv2.rotate(I, rotateCode= cv2.ROTATE_90_CLOCKWISE)
    name = newpath + str(nc) + '.jpg'
    cv2.imwrite(name, c5)
    new_a = [name, cat]
    with open(annotations, 'a') as csvfile:
      csvapp = csv.writer(csvfile)
      csvapp.writerow(new_a)
    
  return nc

In [ ]:
f0 = r'Cat0/*.jpg'
f1 = r'Cat1/*.jpg'
f2 = r'Cat2/*.jpg'
f3 = r'Cat3/*.jpg'
f4 = r'Cat4/*.jpg'

nf0 = r'NCat0//'
nf1 = r'NCat1//'
nf2 = r'NCat2//'
nf3 = r'NCat3//'
nf4 = r'NCat4//'


In [ ]:
#numdata = separate()

nc = 0 # Naming for all new data 
nc = aug(f2, nf2, nc, '2')
print(nc)
nc = aug(f3, nf3, nc, '3')
print(nc)
nc = aug(f4, nf4, nc, '4')
print(nc)
nc = aug(f1, nf1, nc, '1')
print(nc)
nc = aug(f0, nf0, nc, '0')
print(nc)